The following example shows how to generate training data. We want to train the drone flying in y-z plane with different wall distance. 
Notes:
- Turn off disturbance observer to let the data label capture the entirety disturbance force. 
- Run the scenario in simulation_user_guide.ipynb first to make sure the drone does not hit the wall. Adjust wall distance from performance.
- Use the same trajectory in every condition as the same setting in Neural Fly paper.
- Use RandomWaypointsInConstrainedSpace to fly in a constrained 3D space in front of a wall.
- When training disturbance model with inflow model in controller, enable inflow model option in controller.  
- Use baseline disturbacne observer to generate training data.
- Ensure the disturbance model is set to the correct condition, i.e. setting of sinusoidal wind
- restart the ipynb before run


In [1]:
import sys
import os
import numpy as np

# Get the current directory of the notebook
notebook_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import manager
import drone.disturbance_model
import drone.trajectory

def extra_format(value):
    val = abs(value)
    s = f"{val:.1f}".replace('.', '_')
    return f"n{s}" if value < 0 else s

# full time span
# t_data_collection = 120.0
# t_data_collection = 60.0  # training
# t_data_collection = 15.0
t_data_collection = 40.0    # validation
# file_name_template = "test_wind_near_wall_x{}_y{}_z{}_d{}_train_no_bemt"
file_name_template = "test_wind_near_wall_x{}_y{}_z{}_d{}_validation_bemt"
wall_distance_range = [0.1] # in a constrained space, the wall distance naturally varies, so there is no need to set multiple values
# wall_distance_range = [100.0] # in case need to treat as no wall
x_position_error = 0.4  # accounting for controller error to prevent collision to the wall
wall_distance_range = [distance + x_position_error for distance in wall_distance_range]

# full wind range
# wind_range_x = [-5.0, -3.0, -1.0, 0.0]
# # wind_range_y = [-5.0, -3.0, -1.0, 0.0, 1.0, 3.0, 5.0]
# wind_range_y = [0.0]
# wind_range_z = [-5.0, -3.0, -1.0, 0.0, 1.0, 3.0, 5.0]

# limited wind range
# wind_range_x = [-3.0, -1.0, 0.0]
# wind_range_y = [0.0]
# wind_range_z = [-5.0, -3.0, 0.0, 3.0, 5.0]

# validation set
wind_range_x = [-5.0, -2.0, 0.0]
wind_range_y = [0.0]
wind_range_z = [-10.0, -4.0, -1.0, 1.0, 4.0, 10.0]


for wall_distance in wall_distance_range:
    for wind_speed_x in wind_range_x:
        for wind_speed_y in wind_range_y:
            for wind_speed_z in wind_range_z:
                print(f"Running simulation with wind speed ({wind_speed_x}, {wind_speed_y}, {wind_speed_z}) and wall distance {wall_distance}")
                # Initialize the simulation manager
                sim_test = manager.Manager()
                model = drone.disturbance_model.WindEffectNearWall(wall_origin=np.array([-wall_distance, 0, 0]), 
                                                                u_free = np.array([wind_speed_x, wind_speed_y, wind_speed_z]))
                # model = drone.disturbance_model.Free()    # debug only
                # trajectory=drone.trajectory.RandomWaypointsInConstrainedSpace(200, False)
                trajectory = drone.trajectory.Figure8(20)
                sim_test.set_up(disturbance_model=model, trajectory=trajectory)
                sim_test.run(t_data_collection) 
                
                # Save the result
                file_name = file_name_template.format(extra_format(wind_speed_x), 
                                                    extra_format(wind_speed_y),
                                                    extra_format(wind_speed_z),
                                                    extra_format(wall_distance))
                sim_test.save_result(file_name)



[PropellerLookupTable] Reading data from ..\inflow_model\lookup_table\apc_8x6_with_trail.yaml
Running simulation with wind speed (-5.0, 0.0, -10.0) and wall distance 0.5
[PropellerLookupTable] Reading data from ..\inflow_model\lookup_table\apc_8x6_with_trail.yaml
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read fr

Optionally, save the header index into a file for future reference.

In [ ]:
sim_test.logger.generate_data_map()